In [4]:
#  1. we create initial list of urls of articles that we take from news categories (politik, samfund, etc) 
#      web-pages from extrabladet.dk
#  2. this list is then put into a loop to open each of the links and put scraped links from single articles pages. 
#      after all links from #1 are scrapped, combined list of links is put as input file into the loop for next scraping round
#      as a result of each loop round csv file is created 'links_round1', 'links_round2', etc
#  3. combined, deduplicated list of urls is used to scrap data from articles itself: article title, body, author, time of publication
#     'link' of the article is used as unique article id
#      as a result dataframe with 5 columns in stored as 'data_1_4.csv' where 1_4 mean number of round of the loop#2 applied

In [67]:
import pandas as pd
import re

df= pd.read_csv('extrabladet_1974_18082023.csv')

In [68]:
# clean author 
#remove 'Tekst:', take first 2 words = first author

df['a1'] = df['author'].str.replace(',', '')\
                      .str.replace('Af', '')\
                      .str.replace('Ahn', '')\
                      .str.replace('Familiecoach', '')\
                      .str.replace(r'/Mobilsiden\.dk', '')\
                      .str.replace('Ritzau','/ritzau')\
                      .str.replace('For','')
 

df['a2'] = df['a1'].apply(lambda x: ' '.join(x.replace('Tekst:', '')\
                                               .split()[:3]) if isinstance(x, str) else None)

df['a3'] = df['a2'].str.replace(' •', '')\
                   .str.replace(' Husk', '')\
                  .str.replace(' foto:', '')\
                  .str.replace(' Foto:', '')\
                  .str.replace(' Norddjurs', '')\
                  .str.replace(r' \(tekst', '')\
                  .str.replace(' Francorchamps', '')\
                  .str.replace(' Tilsted', '')\
                  .str.replace(r' Mobilsiden\.dk', '')\
                  .str.replace(r' mobilsiden\.dk', '')



C:\Users\lenovo\AppData\Local\Temp\ipykernel_18412\3905431294.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['a1'] = df['author'].str.replace(',', '')\
C:\Users\lenovo\AppData\Local\Temp\ipykernel_18412\3905431294.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['a3'] = df['a2'].str.replace(' •', '')\


In [69]:
u = df['a3'].unique()
sorted_list = sorted(u, key=lambda x: x if isinstance(x, str) else '')

sorted_list

[None,
 '/ritzau',
 '/ritzau /112',
 '/ritzau /Nyheder',
 '/ritzau /Sport',
 '/ritzau /forbrug',
 '/ritzau Fokus',
 '/ritzau/',
 '/ritzau/AFP',
 '/ritzau/NTB',
 '/ritzau/Reuters',
 '/ritzau/TT',
 'Adam Fernandez',
 'Albert Bastian Boysen',
 'Alexander Tørring',
 'Alexander Wøhlk',
 'Allan Lykke',
 'Amalie A. Eriksen',
 'Amalie Louise Larsen',
 'Amanda H. Alberts',
 'Amanda Lagoni',
 'Anders Funder',
 'Anders Larsen',
 'Anders Olsen',
 'Anders Zacho',
 'Andreas Lang Hedegaard',
 'Andreas Munk',
 'Anette Jorsal',
 'Anton Dahl Andersen',
 'Benjamin Krogh',
 'Bertil B. Fruelund',
 'Bjarke Vestesen',
 'Brian Weichardt',
 'Camilia Soltani',
 'Camilla Maria Johansen',
 'Camilla-Maria Behrendt',
 'Carl Emil Nielsen',
 'Caroline Clante',
 'Caroline H. Bilstrup',
 'Carsten Holdum',
 'Carsten Teiner Voty.dk',
 'Casper Hjorth',
 'Cecilie Erland',
 'Cecilie Guldberg',
 'Cecilie Guldberg Jonas',
 'Cecilie Veile',
 'Charlotte Bech Mathiesen',
 'Christian Kloster',
 'Christian Mejdahl',
 'Christian Ri

In [70]:
# Drop rows with no author (NaN)
df.dropna(subset=['a2'], inplace=True)

#check NaN
df['a3'].isna().sum()

0

In [71]:
len(df)

1971

In [72]:
df

,link,title,category,body,time,author,a1,a2,a3
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",2017-12-20T09:48:22+01:00,Niels Friis,Niels Friis,Niels Friis,Niels Friis
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,2015-06-11T16:30:21+02:00,Emil Ingvorsen,Emil Ingvorsen,Emil Ingvorsen,Emil Ingvorsen
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",2021-05-24T06:48:12+02:00,Cecilie Guldberg\n•\n\n\n E...,Cecilie Guldberg\n•\n\n\n E...,Cecilie Guldberg •,Cecilie Guldberg
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",2023-08-11T12:45:42+02:00,Cecilie Guldberg,Cecilie Guldberg,Cecilie Guldberg,Cecilie Guldberg
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",2019-01-19T21:03:26+01:00,Mathias Gram,Mathias Gram,Mathias Gram,Mathias Gram
...,...,...,...,...,...,...,...,...,...
1969,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",2023-08-08T11:45:50+02:00,/ritzau/,/ritzau/,/ritzau/,/ritzau/
1970,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,2023-01-11T14:25:39+01:00,/ritzau/,/ritzau/,/ritzau/,/ritzau/
1971,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",2022-10-02T10:21:10+02:00,Kristian Kornø,Kristian Kornø,Kristian Kornø,Kristian Kornø
1972,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,2023-08-03T13:23:00+02:00,Nanna Bay Madsen,Nanna Bay Madsen,Nanna Bay Madsen,Nanna Bay Madsen


In [73]:
df.drop(columns=['author','a1', 'a2'], inplace=True)
df = df.reset_index(drop=True)
df.rename(columns={'a3': 'author'}, inplace=True)
df

,link,title,category,body,time,author
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",2017-12-20T09:48:22+01:00,Niels Friis
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,2015-06-11T16:30:21+02:00,Emil Ingvorsen
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",2021-05-24T06:48:12+02:00,Cecilie Guldberg
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",2023-08-11T12:45:42+02:00,Cecilie Guldberg
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",2019-01-19T21:03:26+01:00,Mathias Gram
...,...,...,...,...,...,...
1966,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",2023-08-08T11:45:50+02:00,/ritzau/
1967,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,2023-01-11T14:25:39+01:00,/ritzau/
1968,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",2022-10-02T10:21:10+02:00,Kristian Kornø
1969,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,2023-08-03T13:23:00+02:00,Nanna Bay Madsen


In [74]:
#clean time
from datetime import datetime
from datetime import timedelta

# Convert 'time' column to datetime objects with timezone awareness
df['time'] = pd.to_datetime(df['time'], utc=True)

# Add 1 hour to each datetime
df['t1'] = df['time'] + timedelta(hours=1)

# Convert the datetime values to the desired timezone and format as 'YYYY-MM-DD HH:MM'
df['t1'] = df['t1'].dt.strftime('%Y-%m-%d %H:%M')
df

,link,title,category,body,time,author,t1
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",2017-12-20 08:48:22+00:00,Niels Friis,2017-12-20 09:48
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,2015-06-11 14:30:21+00:00,Emil Ingvorsen,2015-06-11 15:30
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",2021-05-24 04:48:12+00:00,Cecilie Guldberg,2021-05-24 05:48
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",2023-08-11 10:45:42+00:00,Cecilie Guldberg,2023-08-11 11:45
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",2019-01-19 20:03:26+00:00,Mathias Gram,2019-01-19 21:03
...,...,...,...,...,...,...,...
1966,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",2023-08-08 09:45:50+00:00,/ritzau/,2023-08-08 10:45
1967,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,2023-01-11 13:25:39+00:00,/ritzau/,2023-01-11 14:25
1968,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",2022-10-02 08:21:10+00:00,Kristian Kornø,2022-10-02 09:21
1969,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,2023-08-03 11:23:00+00:00,Nanna Bay Madsen,2023-08-03 12:23


In [76]:
#df['time'].isna().sum()
df.dropna(subset=['time'], inplace=True)
df

,link,title,category,body,time,author,t1
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",2017-12-20 08:48:22+00:00,Niels Friis,2017-12-20 09:48
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,2015-06-11 14:30:21+00:00,Emil Ingvorsen,2015-06-11 15:30
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",2021-05-24 04:48:12+00:00,Cecilie Guldberg,2021-05-24 05:48
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",2023-08-11 10:45:42+00:00,Cecilie Guldberg,2023-08-11 11:45
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",2019-01-19 20:03:26+00:00,Mathias Gram,2019-01-19 21:03
...,...,...,...,...,...,...,...
1966,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",2023-08-08 09:45:50+00:00,/ritzau/,2023-08-08 10:45
1967,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,2023-01-11 13:25:39+00:00,/ritzau/,2023-01-11 14:25
1968,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",2022-10-02 08:21:10+00:00,Kristian Kornø,2022-10-02 09:21
1969,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,2023-08-03 11:23:00+00:00,Nanna Bay Madsen,2023-08-03 12:23


In [77]:
df['time'].isna().sum()

0

In [78]:
new_df=df.copy()
new_df


,link,title,category,body,time,author,t1
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",2017-12-20 08:48:22+00:00,Niels Friis,2017-12-20 09:48
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,2015-06-11 14:30:21+00:00,Emil Ingvorsen,2015-06-11 15:30
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",2021-05-24 04:48:12+00:00,Cecilie Guldberg,2021-05-24 05:48
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",2023-08-11 10:45:42+00:00,Cecilie Guldberg,2023-08-11 11:45
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",2019-01-19 20:03:26+00:00,Mathias Gram,2019-01-19 21:03
...,...,...,...,...,...,...,...
1966,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",2023-08-08 09:45:50+00:00,/ritzau/,2023-08-08 10:45
1967,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,2023-01-11 13:25:39+00:00,/ritzau/,2023-01-11 14:25
1968,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",2022-10-02 08:21:10+00:00,Kristian Kornø,2022-10-02 09:21
1969,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,2023-08-03 11:23:00+00:00,Nanna Bay Madsen,2023-08-03 12:23


In [79]:
new_df.drop(columns=['time'], inplace=True)
new_df


,link,title,category,body,author,t1
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",Niels Friis,2017-12-20 09:48
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,Emil Ingvorsen,2015-06-11 15:30
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",Cecilie Guldberg,2021-05-24 05:48
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",Cecilie Guldberg,2023-08-11 11:45
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",Mathias Gram,2019-01-19 21:03
...,...,...,...,...,...,...
1966,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",/ritzau/,2023-08-08 10:45
1967,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,/ritzau/,2023-01-11 14:25
1968,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",Kristian Kornø,2022-10-02 09:21
1969,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,Nanna Bay Madsen,2023-08-03 12:23


In [80]:
new_df.rename(columns={'t1':'time'}, inplace=True)
new_df

,link,title,category,body,author,time
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",Niels Friis,2017-12-20 09:48
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,Emil Ingvorsen,2015-06-11 15:30
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",Cecilie Guldberg,2021-05-24 05:48
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",Cecilie Guldberg,2023-08-11 11:45
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",Mathias Gram,2019-01-19 21:03
...,...,...,...,...,...,...
1966,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",/ritzau/,2023-08-08 10:45
1967,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,/ritzau/,2023-01-11 14:25
1968,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",Kristian Kornø,2022-10-02 09:21
1969,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,Nanna Bay Madsen,2023-08-03 12:23


In [106]:
cat_list=pd.read_excel('df_cat.xlsx')

In [107]:
cat_list

,old,new
0,Anden sport,sport
1,Anmeldelser,other
2,Badminton,sport
3,Bilanmeldelser,Underholdning
4,Biler,Underholdning
...,...,...
82,Viborg,other
83,Videnskab,Videnskab
84,Video Underholdning,Underholdning
85,"Film, tv & radio",Underholdning


In [108]:
new_df['category'].isna().sum()


0

In [109]:
# Merge the DataFrames based on the 'category' and 'old' columns
merged_df = new_df.merge(cat_list, left_on='category', right_on='old', how='left')

In [110]:
# Create the 'new_cat' column in the merged DataFrame
merged_df['new_cat'] = merged_df['new']

# Drop unnecessary columns
merged_df.drop(columns=['old', 'new'], inplace=True)

In [111]:
merged_df

,link,title,category,body,author,time,new_cat
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Biler,"Det er ikke sjovt at komme ud til en bil, hvor...",Niels Friis,2017-12-20 09:48,Underholdning
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Krimi,Yahya Hassan har været i en regulær medie-stor...,Emil Ingvorsen,2015-06-11 15:30,krimi
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Dansk politik,"Er du en af de danskere, der kæmper med at få ...",Cecilie Guldberg,2021-05-24 05:48,politik
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Dansk politik,"Med et Venstre i ruiner skulle man måske tro, ...",Cecilie Guldberg,2023-08-11 11:45,politik
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Biler,"16 års abonnement på strøm til elbilen, CO2-ko...",Mathias Gram,2019-01-19 21:03,Underholdning
...,...,...,...,...,...,...,...
1959,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...",/ritzau/,2023-08-08 10:45,Vejret
1960,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,Krig & katastrofer,En selvmordsbombe er eksploderet nær Afghanist...,/ritzau/,2023-01-11 14:25,Udland
1961,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Krimi,"Artilleri-granaten falder med et langstrakt, k...",Kristian Kornø,2022-10-02 09:21,krimi
1962,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Intl. kendte,De seneste dage har den kendte sangerinde Lizz...,Nanna Bay Madsen,2023-08-03 12:23,Underholdning


In [112]:
merged_df['new_cat'].isna().sum()

0

In [113]:
merged_df.drop(columns=['category'], inplace=True)
merged_df

,link,title,body,author,time,new_cat
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,"Det er ikke sjovt at komme ud til en bil, hvor...",Niels Friis,2017-12-20 09:48,Underholdning
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Yahya Hassan har været i en regulær medie-stor...,Emil Ingvorsen,2015-06-11 15:30,krimi
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,"Er du en af de danskere, der kæmper med at få ...",Cecilie Guldberg,2021-05-24 05:48,politik
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,"Med et Venstre i ruiner skulle man måske tro, ...",Cecilie Guldberg,2023-08-11 11:45,politik
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,"16 års abonnement på strøm til elbilen, CO2-ko...",Mathias Gram,2019-01-19 21:03,Underholdning
...,...,...,...,...,...,...
1959,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,"Uvejret, der har ramt Danmark, har resulteret ...",/ritzau/,2023-08-08 10:45,Vejret
1960,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,En selvmordsbombe er eksploderet nær Afghanist...,/ritzau/,2023-01-11 14:25,Udland
1961,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,"Artilleri-granaten falder med et langstrakt, k...",Kristian Kornø,2022-10-02 09:21,krimi
1962,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,De seneste dage har den kendte sangerinde Lizz...,Nanna Bay Madsen,2023-08-03 12:23,Underholdning


In [114]:
merged_df.rename(columns={'new_cat':'category'}, inplace=True)
merged_df

,link,title,body,author,time,category
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,"Det er ikke sjovt at komme ud til en bil, hvor...",Niels Friis,2017-12-20 09:48,Underholdning
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Yahya Hassan har været i en regulær medie-stor...,Emil Ingvorsen,2015-06-11 15:30,krimi
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,"Er du en af de danskere, der kæmper med at få ...",Cecilie Guldberg,2021-05-24 05:48,politik
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,"Med et Venstre i ruiner skulle man måske tro, ...",Cecilie Guldberg,2023-08-11 11:45,politik
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,"16 års abonnement på strøm til elbilen, CO2-ko...",Mathias Gram,2019-01-19 21:03,Underholdning
...,...,...,...,...,...,...
1959,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,"Uvejret, der har ramt Danmark, har resulteret ...",/ritzau/,2023-08-08 10:45,Vejret
1960,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,En selvmordsbombe er eksploderet nær Afghanist...,/ritzau/,2023-01-11 14:25,Udland
1961,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,"Artilleri-granaten falder med et langstrakt, k...",Kristian Kornø,2022-10-02 09:21,krimi
1962,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,De seneste dage har den kendte sangerinde Lizz...,Nanna Bay Madsen,2023-08-03 12:23,Underholdning


In [115]:
#clean 'body' of the article
import re

# remove words ending with 'GemGemt' (this is tag to save articles for further reading)
def remove_gemgemt(word):
    if isinstance(word, str):  # Check if the input is a string
        return re.sub(r'\w*GemGemt$', '', word)
    return word

# Apply the function to the 'body' column
merged_df['body_clean'] = merged_df['body'].apply(remove_gemgemt)

merged_df

,link,title,body,author,time,category,body_clean
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,"Det er ikke sjovt at komme ud til en bil, hvor...",Niels Friis,2017-12-20 09:48,Underholdning,"Det er ikke sjovt at komme ud til en bil, hvor..."
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Yahya Hassan har været i en regulær medie-stor...,Emil Ingvorsen,2015-06-11 15:30,krimi,Yahya Hassan har været i en regulær medie-stor...
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,"Er du en af de danskere, der kæmper med at få ...",Cecilie Guldberg,2021-05-24 05:48,politik,"Er du en af de danskere, der kæmper med at få ..."
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,"Med et Venstre i ruiner skulle man måske tro, ...",Cecilie Guldberg,2023-08-11 11:45,politik,"Med et Venstre i ruiner skulle man måske tro, ..."
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,"16 års abonnement på strøm til elbilen, CO2-ko...",Mathias Gram,2019-01-19 21:03,Underholdning,"16 års abonnement på strøm til elbilen, CO2-ko..."
...,...,...,...,...,...,...,...
1959,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,"Uvejret, der har ramt Danmark, har resulteret ...",/ritzau/,2023-08-08 10:45,Vejret,"Uvejret, der har ramt Danmark, har resulteret ..."
1960,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,En selvmordsbombe er eksploderet nær Afghanist...,/ritzau/,2023-01-11 14:25,Udland,En selvmordsbombe er eksploderet nær Afghanist...
1961,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,"Artilleri-granaten falder med et langstrakt, k...",Kristian Kornø,2022-10-02 09:21,krimi,"Artilleri-granaten falder med et langstrakt, k..."
1962,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,De seneste dage har den kendte sangerinde Lizz...,Nanna Bay Madsen,2023-08-03 12:23,Underholdning,De seneste dage har den kendte sangerinde Lizz...


In [116]:
merged_df['body_1'] = merged_df['body_clean'].replace(['\\n', '\\xa0'], ['', ''], regex=True)
merged_df

,link,title,body,author,time,category,body_clean,body_1
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,"Det er ikke sjovt at komme ud til en bil, hvor...",Niels Friis,2017-12-20 09:48,Underholdning,"Det er ikke sjovt at komme ud til en bil, hvor...","Det er ikke sjovt at komme ud til en bil, hvor..."
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Yahya Hassan har været i en regulær medie-stor...,Emil Ingvorsen,2015-06-11 15:30,krimi,Yahya Hassan har været i en regulær medie-stor...,Yahya Hassan har været i en regulær medie-stor...
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,"Er du en af de danskere, der kæmper med at få ...",Cecilie Guldberg,2021-05-24 05:48,politik,"Er du en af de danskere, der kæmper med at få ...","Er du en af de danskere, der kæmper med at få ..."
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,"Med et Venstre i ruiner skulle man måske tro, ...",Cecilie Guldberg,2023-08-11 11:45,politik,"Med et Venstre i ruiner skulle man måske tro, ...","Med et Venstre i ruiner skulle man måske tro, ..."
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,"16 års abonnement på strøm til elbilen, CO2-ko...",Mathias Gram,2019-01-19 21:03,Underholdning,"16 års abonnement på strøm til elbilen, CO2-ko...","16 års abonnement på strøm til elbilen, CO2-ko..."
...,...,...,...,...,...,...,...,...
1959,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,"Uvejret, der har ramt Danmark, har resulteret ...",/ritzau/,2023-08-08 10:45,Vejret,"Uvejret, der har ramt Danmark, har resulteret ...","Uvejret, der har ramt Danmark, har resulteret ..."
1960,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,En selvmordsbombe er eksploderet nær Afghanist...,/ritzau/,2023-01-11 14:25,Udland,En selvmordsbombe er eksploderet nær Afghanist...,En selvmordsbombe er eksploderet nær Afghanist...
1961,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,"Artilleri-granaten falder med et langstrakt, k...",Kristian Kornø,2022-10-02 09:21,krimi,"Artilleri-granaten falder med et langstrakt, k...","Artilleri-granaten falder med et langstrakt, k..."
1962,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,De seneste dage har den kendte sangerinde Lizz...,Nanna Bay Madsen,2023-08-03 12:23,Underholdning,De seneste dage har den kendte sangerinde Lizz...,De seneste dage har den kendte sangerinde Lizz...


In [117]:
merged_df.drop(columns=['body','body_clean'], inplace=True)
merged_df

,link,title,author,time,category,body_1
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Niels Friis,2017-12-20 09:48,Underholdning,"Det er ikke sjovt at komme ud til en bil, hvor..."
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Emil Ingvorsen,2015-06-11 15:30,krimi,Yahya Hassan har været i en regulær medie-stor...
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Cecilie Guldberg,2021-05-24 05:48,politik,"Er du en af de danskere, der kæmper med at få ..."
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Cecilie Guldberg,2023-08-11 11:45,politik,"Med et Venstre i ruiner skulle man måske tro, ..."
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Mathias Gram,2019-01-19 21:03,Underholdning,"16 års abonnement på strøm til elbilen, CO2-ko..."
...,...,...,...,...,...,...
1959,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,/ritzau/,2023-08-08 10:45,Vejret,"Uvejret, der har ramt Danmark, har resulteret ..."
1960,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,/ritzau/,2023-01-11 14:25,Udland,En selvmordsbombe er eksploderet nær Afghanist...
1961,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Kristian Kornø,2022-10-02 09:21,krimi,"Artilleri-granaten falder med et langstrakt, k..."
1962,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Nanna Bay Madsen,2023-08-03 12:23,Underholdning,De seneste dage har den kendte sangerinde Lizz...


In [118]:
merged_df.rename(columns={'body_1':'body'}, inplace=True)
merged_df

,link,title,author,time,category,body
0,https://ekstrabladet.dk/biler/goer-du-det-her-...,Gør du det her om morgenen? Det er ulovligt - ...,Niels Friis,2017-12-20 09:48,Underholdning,"Det er ikke sjovt at komme ud til en bil, hvor..."
1,https://ekstrabladet.dk/krimi/yahya-i-ny-balla...,Yahya i ny ballade: I klammeri med bartender,Emil Ingvorsen,2015-06-11 15:30,krimi,Yahya Hassan har været i en regulær medie-stor...
2,https://ekstrabladet.dk/nyheder/politik/danskp...,Kalder minister i samråd: - Det gør mig rasende,Cecilie Guldberg,2021-05-24 05:48,politik,"Er du en af de danskere, der kæmper med at få ..."
3,https://ekstrabladet.dk/nyheder/politik/danskp...,Konservativ krise: Papes problemer står i kø,Cecilie Guldberg,2023-08-11 11:45,politik,"Med et Venstre i ruiner skulle man måske tro, ..."
4,https://ekstrabladet.dk/biler/400-danskere-paa...,400 danskere på venteliste: Hypet elbil får ny...,Mathias Gram,2019-01-19 21:03,Underholdning,"16 års abonnement på strøm til elbilen, CO2-ko..."
...,...,...,...,...,...,...
1959,https://ekstrabladet.dk/nyheder/samfund/vejret...,Utætte tage og flyvende trampoliner: Uvejr giv...,/ritzau/,2023-08-08 10:45,Vejret,"Uvejret, der har ramt Danmark, har resulteret ..."
1960,https://ekstrabladet.dk/nyheder/krigogkatastro...,Bombe eksploderer i det centrale Kabul med man...,/ritzau/,2023-01-11 14:25,Udland,En selvmordsbombe er eksploderet nær Afghanist...
1961,https://ekstrabladet.dk/krimi/mike-slaas-i-ukr...,Mike slås i Ukraine: Putins danske mareridt,Kristian Kornø,2022-10-02 09:21,krimi,"Artilleri-granaten falder med et langstrakt, k..."
1962,https://ekstrabladet.dk/underholdning/udlandke...,Efter voldsomme anklager: Nu bryder hun tavsheden,Nanna Bay Madsen,2023-08-03 12:23,Underholdning,De seneste dage har den kendte sangerinde Lizz...


In [119]:
# if article is only available via subscription 'body' < 100

merged_df['num_of_words'] = merged_df['body'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
merged_df = merged_df.sort_values(by='num_of_words', ascending=True)

merged_df

,link,title,author,time,category,body,num_of_words
1583,https://ekstrabladet.dk/underholdning/pernille...,Pernille fra Bagedysten: Pumpede mænd og store...,Pernille Christensen,2015-10-23 11:31,Underholdning,NaN,0
558,https://ekstrabladet.dk/side9/jeanett/9894235,"Jeanett, 25 år og fra Nakskov",Ekstra Bladet,2023-08-17 21:00,other,NaN,0
758,https://ekstrabladet.dk/nyheder/samfund/reimar...,"Reimar om Sarah Louise: Nå, så hun er kun dum ...",Lotte Reimar,2017-03-24 20:02,samfund,NaN,0
1547,https://ekstrabladet.dk/sport/fodbold/se-fcks-...,Se FCK's Sparta-brag her,Casper Hjorth,2023-08-12 06:40,sport,NaN,0
823,https://ekstrabladet.dk/nyheder/politik/oproer...,Oprør fra vandkanten: Staten opstiller 200 met...,Per Mathiessen,2018-07-15 07:09,politik,NaN,0
...,...,...,...,...,...,...,...
674,https://ekstrabladet.dk/nyheder/samfund/famili...,Familie splittet: Mor til baby smidt ud,Cecilie Guldberg,2023-03-08 12:14,samfund,På første sal i en baggårdslejlighed i Helsing...,2366
278,https://ekstrabladet.dk/nationen/langer-ud-toe...,Langer ud: Tøsedrenge,Thomas Harder,2021-09-12 21:34,sport,- FN´s klimapanel (IPCC) publicerede i august ...,2398
291,https://ekstrabladet.dk/nyheder/samfund/smidt-...,Smidt ud efter regel-koks: Forsøgte at tage si...,Cecilie Guldberg,2023-03-02 07:52,samfund,"Klokken er kvart over to om natten, og thailan...",2399
972,https://ekstrabladet.dk/nationen/helt-haabloes...,Helt håbløst: Projekt til 200 milliarder søger...,Thomas Harder,2021-06-01 17:16,sport,- Vi - Ib og Søren – skrev i Ekstra Bladets e-...,2692


In [120]:
# drop rows with no values for 'body'
merged_df.dropna(subset=['body'], inplace=True)
merged_df

,link,title,author,time,category,body,num_of_words
1102,https://ekstrabladet.dk/nyheder/jordskaelv-rys...,Jordskælv ryster hovedstad,/ritzau/,2023-08-17 18:35,samfund,Opdateres ...,2
1341,https://ekstrabladet.dk/underholdning/livsstil...,Kom med indenfor: Det dyreste nogensinde,Peter Roelsgaard,2023-08-07 09:35,Underholdning,"Den Internationale Rumstation er det dyreste, ...",21
1535,https://ekstrabladet.dk/underholdning/kultur/c...,Copenhagen Pride: Fra bankerot til sponsor-stop,Peter Roelsgaard,2023-08-13 06:10,Underholdning,Oplev Copenhagen Pride i dette gigantiske gall...,21
1644,https://ekstrabladet.dk/nyheder/samfund/atomva...,Atomvåben og missiler: Så stærk er Nordkoreas hær,Alexander Tørring,2021-09-30 14:37,samfund,Hvem kan Nordkorea potentielt ramme med missil...,22
142,https://ekstrabladet.dk/krimi/saadan-blev-23-t...,"Sådan blev 2,3 tons hash fragtet fra landsby t...",Bjarke Vestesen,2023-08-13 06:11,krimi,Tre byer på Sjælland spiller afgørende roller ...,24
...,...,...,...,...,...,...,...
674,https://ekstrabladet.dk/nyheder/samfund/famili...,Familie splittet: Mor til baby smidt ud,Cecilie Guldberg,2023-03-08 12:14,samfund,På første sal i en baggårdslejlighed i Helsing...,2366
278,https://ekstrabladet.dk/nationen/langer-ud-toe...,Langer ud: Tøsedrenge,Thomas Harder,2021-09-12 21:34,sport,- FN´s klimapanel (IPCC) publicerede i august ...,2398
291,https://ekstrabladet.dk/nyheder/samfund/smidt-...,Smidt ud efter regel-koks: Forsøgte at tage si...,Cecilie Guldberg,2023-03-02 07:52,samfund,"Klokken er kvart over to om natten, og thailan...",2399
972,https://ekstrabladet.dk/nationen/helt-haabloes...,Helt håbløst: Projekt til 200 milliarder søger...,Thomas Harder,2021-06-01 17:16,sport,- Vi - Ib og Søren – skrev i Ekstra Bladets e-...,2692


In [129]:
# check empty values 
nan_title = merged_df['title'].isna().sum()
nan_author = merged_df['author'].isna().sum()
nan_time = merged_df['time'].isna().sum()
nan_category = merged_df['category'].isna().sum()
nan_body = merged_df['body'].isna().sum()

print(f'No values title: {nan_title}')
print(f'No values author: {nan_author}')
print(f'No values time: {nan_time}')
print(f'No values category: {nan_category}')
print(f'No values body: {nan_body}')

No values title: 0
No values author: 0
No values time: 0
No values category: 0
No values body: 0


In [128]:
rows= merged_df[merged_df['title'].isna()]
rows

,link,title,author,time,category,body,num_of_words


In [130]:
merged_df.reset_index(drop=True)
copy= merged_df

In [132]:
merged_df

,link,title,author,time,category,body,num_of_words
1102,https://ekstrabladet.dk/nyheder/jordskaelv-rys...,Jordskælv ryster hovedstad,/ritzau/,2023-08-17 18:35,samfund,Opdateres ...,2
1341,https://ekstrabladet.dk/underholdning/livsstil...,Kom med indenfor: Det dyreste nogensinde,Peter Roelsgaard,2023-08-07 09:35,Underholdning,"Den Internationale Rumstation er det dyreste, ...",21
1535,https://ekstrabladet.dk/underholdning/kultur/c...,Copenhagen Pride: Fra bankerot til sponsor-stop,Peter Roelsgaard,2023-08-13 06:10,Underholdning,Oplev Copenhagen Pride i dette gigantiske gall...,21
1644,https://ekstrabladet.dk/nyheder/samfund/atomva...,Atomvåben og missiler: Så stærk er Nordkoreas hær,Alexander Tørring,2021-09-30 14:37,samfund,Hvem kan Nordkorea potentielt ramme med missil...,22
142,https://ekstrabladet.dk/krimi/saadan-blev-23-t...,"Sådan blev 2,3 tons hash fragtet fra landsby t...",Bjarke Vestesen,2023-08-13 06:11,krimi,Tre byer på Sjælland spiller afgørende roller ...,24
...,...,...,...,...,...,...,...
674,https://ekstrabladet.dk/nyheder/samfund/famili...,Familie splittet: Mor til baby smidt ud,Cecilie Guldberg,2023-03-08 12:14,samfund,På første sal i en baggårdslejlighed i Helsing...,2366
278,https://ekstrabladet.dk/nationen/langer-ud-toe...,Langer ud: Tøsedrenge,Thomas Harder,2021-09-12 21:34,sport,- FN´s klimapanel (IPCC) publicerede i august ...,2398
291,https://ekstrabladet.dk/nyheder/samfund/smidt-...,Smidt ud efter regel-koks: Forsøgte at tage si...,Cecilie Guldberg,2023-03-02 07:52,samfund,"Klokken er kvart over to om natten, og thailan...",2399
972,https://ekstrabladet.dk/nationen/helt-haabloes...,Helt håbløst: Projekt til 200 milliarder søger...,Thomas Harder,2021-06-01 17:16,sport,- Vi - Ib og Søren – skrev i Ekstra Bladets e-...,2692


In [137]:
copy.reset_index(inplace=True)

copy

,index,link,title,author,time,category,body,num_of_words
0,1102,https://ekstrabladet.dk/nyheder/jordskaelv-rys...,Jordskælv ryster hovedstad,/ritzau/,2023-08-17 18:35,samfund,Opdateres ...,2
1,1341,https://ekstrabladet.dk/underholdning/livsstil...,Kom med indenfor: Det dyreste nogensinde,Peter Roelsgaard,2023-08-07 09:35,Underholdning,"Den Internationale Rumstation er det dyreste, ...",21
2,1535,https://ekstrabladet.dk/underholdning/kultur/c...,Copenhagen Pride: Fra bankerot til sponsor-stop,Peter Roelsgaard,2023-08-13 06:10,Underholdning,Oplev Copenhagen Pride i dette gigantiske gall...,21
3,1644,https://ekstrabladet.dk/nyheder/samfund/atomva...,Atomvåben og missiler: Så stærk er Nordkoreas hær,Alexander Tørring,2021-09-30 14:37,samfund,Hvem kan Nordkorea potentielt ramme med missil...,22
4,142,https://ekstrabladet.dk/krimi/saadan-blev-23-t...,"Sådan blev 2,3 tons hash fragtet fra landsby t...",Bjarke Vestesen,2023-08-13 06:11,krimi,Tre byer på Sjælland spiller afgørende roller ...,24
...,...,...,...,...,...,...,...,...
1953,674,https://ekstrabladet.dk/nyheder/samfund/famili...,Familie splittet: Mor til baby smidt ud,Cecilie Guldberg,2023-03-08 12:14,samfund,På første sal i en baggårdslejlighed i Helsing...,2366
1954,278,https://ekstrabladet.dk/nationen/langer-ud-toe...,Langer ud: Tøsedrenge,Thomas Harder,2021-09-12 21:34,sport,- FN´s klimapanel (IPCC) publicerede i august ...,2398
1955,291,https://ekstrabladet.dk/nyheder/samfund/smidt-...,Smidt ud efter regel-koks: Forsøgte at tage si...,Cecilie Guldberg,2023-03-02 07:52,samfund,"Klokken er kvart over to om natten, og thailan...",2399
1956,972,https://ekstrabladet.dk/nationen/helt-haabloes...,Helt håbløst: Projekt til 200 milliarder søger...,Thomas Harder,2021-06-01 17:16,sport,- Vi - Ib og Søren – skrev i Ekstra Bladets e-...,2692


In [138]:
copy.drop(columns=['index'], inplace=True)
copy

,link,title,author,time,category,body,num_of_words
0,https://ekstrabladet.dk/nyheder/jordskaelv-rys...,Jordskælv ryster hovedstad,/ritzau/,2023-08-17 18:35,samfund,Opdateres ...,2
1,https://ekstrabladet.dk/underholdning/livsstil...,Kom med indenfor: Det dyreste nogensinde,Peter Roelsgaard,2023-08-07 09:35,Underholdning,"Den Internationale Rumstation er det dyreste, ...",21
2,https://ekstrabladet.dk/underholdning/kultur/c...,Copenhagen Pride: Fra bankerot til sponsor-stop,Peter Roelsgaard,2023-08-13 06:10,Underholdning,Oplev Copenhagen Pride i dette gigantiske gall...,21
3,https://ekstrabladet.dk/nyheder/samfund/atomva...,Atomvåben og missiler: Så stærk er Nordkoreas hær,Alexander Tørring,2021-09-30 14:37,samfund,Hvem kan Nordkorea potentielt ramme med missil...,22
4,https://ekstrabladet.dk/krimi/saadan-blev-23-t...,"Sådan blev 2,3 tons hash fragtet fra landsby t...",Bjarke Vestesen,2023-08-13 06:11,krimi,Tre byer på Sjælland spiller afgørende roller ...,24
...,...,...,...,...,...,...,...
1953,https://ekstrabladet.dk/nyheder/samfund/famili...,Familie splittet: Mor til baby smidt ud,Cecilie Guldberg,2023-03-08 12:14,samfund,På første sal i en baggårdslejlighed i Helsing...,2366
1954,https://ekstrabladet.dk/nationen/langer-ud-toe...,Langer ud: Tøsedrenge,Thomas Harder,2021-09-12 21:34,sport,- FN´s klimapanel (IPCC) publicerede i august ...,2398
1955,https://ekstrabladet.dk/nyheder/samfund/smidt-...,Smidt ud efter regel-koks: Forsøgte at tage si...,Cecilie Guldberg,2023-03-02 07:52,samfund,"Klokken er kvart over to om natten, og thailan...",2399
1956,https://ekstrabladet.dk/nationen/helt-haabloes...,Helt håbløst: Projekt til 200 milliarder søger...,Thomas Harder,2021-06-01 17:16,sport,- Vi - Ib og Søren – skrev i Ekstra Bladets e-...,2692


In [141]:
copy.iloc[100]

link            https://ekstrabladet.dk/underholdning/droemmen...
title               Drømmen ændrede sig: Micki tilbage på skærmen
author                                       Pernille Christensen
time                                             2016-09-14 10:08
category                                            Underholdning
body            I aften indtager ti forventningsfulde amatørba...
num_of_words                                                   55
Name: 100, dtype: object

In [139]:
# Drop rows where 'num_of_words' < 100
final = copy[copy['num_of_words'] >= 100]


In [272]:
new= final.reset_index(drop=True)
new

,link,title,author,time,category,body,num_of_words
0,https://ekstrabladet.dk/krimi/bowlinghal-tilta...,Bowlinghal tiltalt for uagtsomt manddrab,Signe Søgaard,2023-08-15 14:12,krimi,I sommeren sidste år mistede en 29-årig mand l...,100
1,https://ekstrabladet.dk/krimi/boederegn-181-bi...,Bøderegn: 181 bilister gået i fælden,Lasse Emil,2023-08-15 18:41,krimi,"Skoleferien er ovre, og bilisterne er tilbage ...",102
2,https://ekstrabladet.dk/krimi/efterlyser-39-aa...,Efterlyser 39-årig: Mistænkt for trusler,Petra Akullu,2023-08-15 08:35,krimi,I juli skulle 39-årige Christian Peder Majlund...,106
3,https://ekstrabladet.dk/krimi/alvorlig-ulykke-...,Alvorlig ulykke: Mand i livsfare,Lasse Emil,2023-08-14 19:35,krimi,En venstresvingsulykke i Hjerting ved Esbjerg ...,107
4,https://ekstrabladet.dk/krimi/moedt-af-chok-de...,Fik et chok: - Der var vand overalt,Signe Søgaard,2023-08-15 13:43,krimi,I weekenden sneg ukendte gerningspersoner sig ...,112
...,...,...,...,...,...,...,...
424,https://ekstrabladet.dk/nyheder/politik/danskp...,Bjarne er færdig med danske gakgak-regler: - D...,Cecilie Guldberg,2022-10-10 12:22,politik,Udlændinge- og integrationsminister Kaare Dybv...,2099
425,https://ekstrabladet.dk/nyheder/samfund/2200-p...,2200 papirer var ikke bevis nok: Jeg er rystet...,Cecilie Guldberg,2021-06-15 10:24,samfund,I 2017 mødte danske Lasse Dalby Jensen og aser...,2190
426,https://ekstrabladet.dk/nyheder/samfund/havde-...,Havde alt i Danmark: Så blev han smidt ud og p...,Emmely Smith,2022-08-06 14:26,samfund,Har du selv oplevet at komme i klemme i regler...,2203
427,https://ekstrabladet.dk/nyheder/samfund/famili...,Familie splittet: Mor til baby smidt ud,Cecilie Guldberg,2023-03-08 12:14,samfund,På første sal i en baggårdslejlighed i Helsing...,2366


In [274]:
new.to_csv('extrabladet_429articles.csv')